# Дискретное преобразование Фурье

Кобыжев Александр, группа 3530901/80202

### Упражнение 7.2

In [1]:
from __future__ import print_function, division

import thinkdsp
import thinkplot

import numpy as np

import warnings
warnings.filterwarnings('ignore')

PI2 = 2 * np.pi

np.set_printoptions(precision=3, suppress=True)
%matplotlib inline

Возьмём небольшой реальный сигнал и вычислим его БПФ:

In [2]:
ys = [-0.4, 0.2, 0.8, -0.2]
hs = np.fft.fft(ys)
print(hs)

[ 0.4+0.j  -1.2-0.4j  0.4+0.j  -1.2+0.4j]


Напишем функцию для БПФ:

In [3]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

Удостоверимся, что эта реализация даёт тот же результат, что и `np.fft.fft`.

In [4]:
hs2 = dft(ys)
print(sum(abs(hs - hs2)))

7.507415150515407e-16


Чтобы сделать рекурсивное БПФ, я начну с версии, которая разбивает входной массив и использует `np.fft.fft` для вычисления БПФ половин.

In [5]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Теперь мы также должны получить такие же результаты.

In [6]:
hs3 = fft_norec(ys)
print(sum(abs(hs - hs3)))

0.0


Теперь мы можем заменить `np.fft.fft` рекурсивными вызовами и добавить базовый случай:

In [7]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Результаты снова совпадают:

In [8]:
hs4 = fft(ys)
print(sum(abs(hs - hs4)))

2.220446049250313e-16


Эта реализация БПФ требует времени и пространства, пропорционального $nlogn$. и это тратит время на создание и копирование массивов. Его можно улучшить, чтобы он работал «на месте», но в этом случае он не требует дополнительного места и тратит меньше времени на накладные расходы.